In [14]:
import os
from data.source.pg_experiment import get_pg_experiment_dataframe
import polars as pl

from models.SimplifiedLightweightCNN import SimplifiedLightweightCNN
%load_ext autoreload
%autoreload 1
%aimport models.SimplifiedLightweightCNN
from models.SimpleCNN_v2 import train, evaluate
import torch
from torch.utils.data import DataLoader
import torch.nn as nn
from path import RESULT_DIRECTORY
import wandb

os.environ["KMP_DUPLICATE_LIB_OK"] = "TRUE"

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [ ]:
df_pron, df_tone = get_pg_experiment_dataframe(".ogg")
df_stageI_polish = df_pron.filter((pl.col("stage") == 1) & (pl.col("mother") == "polish"))
df_other_lang = df_pron.filter(
    (pl.col("stage") == 1) & ~(pl.col("mother") == "polish")
)

df_stageI_polish

In [ ]:
import polars as pl
import numpy as np
from typing import Tuple

def stratified_split(df: pl.DataFrame, label_col: str, train_frac=0.8, val_frac=0.1, seed=42) -> Tuple[pl.DataFrame, pl.DataFrame, pl.DataFrame]:
    classes = df.select(label_col).unique().to_series()
    train_rows, val_rows, test_rows = [], [], []

    rng = np.random.RandomState(seed)

    for cls in classes:
        class_df = df.filter(pl.col(label_col) == cls)
        n = class_df.height
        indices = rng.permutation(n)

        train_end = int(train_frac * n)
        val_end = int((train_frac + val_frac) * n)

        train_rows.append(class_df[indices[:train_end]])
        val_rows.append(class_df[indices[train_end:val_end]])
        test_rows.append(class_df[indices[val_end:]])

    train_df = pl.concat(train_rows)
    val_df = pl.concat(val_rows)
    test_df = pl.concat(test_rows)

    return train_df, val_df, test_df


In [ ]:
from dataset import Cast, TorchDataset
from pytorch_dataloader import build_collate_fn, PaddingCollate, DefaultCollate
from functools import partial

from transformation import TorchVadLogMelSpec, TorchVadMFCC
from develop import reload_function

reload_function(TorchVadMFCC)

TRAIN_SPLIT = 0.6
VAL_SPLIT = 0.2
TEST_SPLIT = 1 - TRAIN_SPLIT - VAL_SPLIT
train_pl, val_pl, test_pl = stratified_split(df_stageI_polish, label_col="value", train_frac=TRAIN_SPLIT, val_frac=VAL_SPLIT)

to_dataset = lambda dataframe: TorchDataset(
    Cast(dataframe.get_column("rec_path"), TorchVadMFCC(n_mfcc=20)), 
    Cast(dataframe.get_column("value"), lambda x: torch.tensor(x).float()),
)

collate_fn = build_collate_fn(
    PaddingCollate(mode="SET_MAX_LEN", max_len=128),
    DefaultCollate()
)
dataset_train = to_dataset(train_pl)
dataset_val = to_dataset(val_pl)
dataset_test = to_dataset(test_pl)

In [9]:
from pytorch_dataloader import MemoryLoadedDataLoader

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

train_loader = DataLoader(dataset_train, batch_size=16, shuffle=True, collate_fn=collate_fn, num_workers=4)
val_loader = DataLoader(dataset_val, batch_size=16, shuffle=False, collate_fn=collate_fn, num_workers=4)
test_loader = DataLoader(dataset_test, batch_size=16, shuffle=False, collate_fn=collate_fn, num_workers=4)

train_loader = MemoryLoadedDataLoader(train_loader, device=device)
print("Loaded train loader into memory")
val_loader = MemoryLoadedDataLoader(val_loader, device=device)
print("Loaded validation loader into memory")

Loaded train loader into memory
Loaded validation loader into memory


In [9]:
model = SimplifiedLightweightCNN(input_channels=1, num_classes=1, dropout_rate=0.3)
model.eval()

SimplifiedLightweightCNN(
  (features): Sequential(
    (0): Conv2d(1, 32, kernel_size=(5, 5), stride=(1, 1), padding=(2, 2))
    (1): BatchNorm2d(32, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (2): ReLU()
    (3): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (4): Dropout2d(p=0.1, inplace=False)
    (5): Conv2d(32, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (6): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (7): ReLU()
    (8): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (9): Dropout2d(p=0.2, inplace=False)
    (10): Conv2d(64, 96, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (11): BatchNorm2d(96, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (12): ReLU()
    (13): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (14): Dropout2d(p=0.3, inplace=False)
    (15): AdaptiveAvgPool2d(o

In [20]:
# Model variables definition.
pth = "SimplifiedLightweightCNN.pth"
lr = 5e-4  # Reduce from 1e-3
epochs = 100
model = model.to(device)
optimizer = torch.optim.Adam(model.parameters(), lr=lr, weight_decay=1e-4)  # Add L2 regularization
# Add learning rate scheduler
scheduler = torch.optim.lr_scheduler.ReduceLROnPlateau(
    optimizer, mode='min', factor=0.5, patience=5
)
criterion = nn.BCELoss()

# Start a new wandb run to track this script.
run = wandb.init(
    # name of the run
    name="SimplifiedLightweightCNN-run-3",
    config={
        "Name": 'SimplifiedLightweightCNN',
        "learning_rate": lr,
        "optimizer": "Adam",
        "criterion": "BCELoss",
        "architecture": "SimplifiedLightweightCNN",
        "dataset": "Stage-I-only-polish",
        "train_val_test(%)": f'{train_split}-{val_split}-{test_split}',
        "epochs": epochs,
    },
)

# Training loop
for epoch in range(epochs):
    train_loss, train_acc = train(model, train_loader, optimizer, criterion, device)
    val_loss, val_acc = evaluate(model, val_loader, criterion, device)
    # Update learning rate
    scheduler.step(val_loss)
    # Logging the metadata for each epoch so that the charts can be generated on the dashboard
    run.log({"train_acc": train_acc, "train_loss": train_loss, "val_acc": val_acc, "val_loss": val_loss, })
    print(
        f"Epoch {epoch + 1}, Train Loss: {train_loss:.4f}, Train Acc: {train_acc:.4f}, Val Loss: {val_loss:.4f}, Val Acc: {val_acc:.4f}")

run.log({"model_eval": model.eval()})
# Saving the model to pth and adding it to the artifacts of the run, there is 5GB of memory on wandb, so we should be fine.
torch.save(model.state_dict(), os.path.join(RESULT_DIRECTORY, pth))
artifact = wandb.Artifact("SimplifiedLightweightCNN-model", type="model")
artifact.add_file(os.path.join(RESULT_DIRECTORY, pth))
run.log_artifact(artifact)

# Finish the run so it gets sent to the remote. You can discover the run right after that on the dashboard.
run.finish()


Epoch 1, Train Loss: 0.6823, Train Acc: 0.5734, Val Loss: 0.6775, Val Acc: 0.5820
Epoch 2, Train Loss: 0.6801, Train Acc: 0.5816, Val Loss: 0.6754, Val Acc: 0.5864
Epoch 3, Train Loss: 0.6773, Train Acc: 0.5839, Val Loss: 0.6745, Val Acc: 0.5899
Epoch 4, Train Loss: 0.6770, Train Acc: 0.5871, Val Loss: 0.6736, Val Acc: 0.5890
Epoch 5, Train Loss: 0.6762, Train Acc: 0.5877, Val Loss: 0.6727, Val Acc: 0.5899
Epoch 6, Train Loss: 0.6749, Train Acc: 0.5862, Val Loss: 0.6712, Val Acc: 0.5916
Epoch 7, Train Loss: 0.6760, Train Acc: 0.5880, Val Loss: 0.6705, Val Acc: 0.5960
Epoch 8, Train Loss: 0.6735, Train Acc: 0.5955, Val Loss: 0.6689, Val Acc: 0.5942
Epoch 9, Train Loss: 0.6703, Train Acc: 0.5976, Val Loss: 0.6674, Val Acc: 0.6038
Epoch 10, Train Loss: 0.6675, Train Acc: 0.5993, Val Loss: 0.6680, Val Acc: 0.5995
Epoch 11, Train Loss: 0.6679, Train Acc: 0.6008, Val Loss: 0.6668, Val Acc: 0.6012
Epoch 12, Train Loss: 0.6694, Train Acc: 0.5961, Val Loss: 0.6672, Val Acc: 0.6021
Epoch 13, Tra

train_acc,▁▁▁▁▂▂▃▂▃▃▃▄▄▄▄▅▆▅▆▆▇▇▇▆▆▆▆▇▇█▇▇▇█▇▇████
train_loss,████▇▇▇▇▇▇▆▆▆▆▆▆▅▅▅▅▄▄▃▃▃▃▂▂▂▂▁▂▂▂▂▁▁▁▂▂
val_acc,▁▂▂▂▂▃▃▄▃▄▅▄▄▄▅▅▅▅▅▆▆▇▇▇▆█▆▇▇▆██▇█▇▇▇▇▇▇
val_loss,███▇▇▇▆▆▅▅▅▄▄▃▃▃▂▂▁▂▂▂▁▂▂▁▁▁▁▁▁▁▂▂▁▂▁▁▁▁
model_eval,SimplifiedLightweigh...
train_acc,0.66938
train_loss,0.60965
val_acc,0.63787
val_loss,0.64068


In [11]:
# device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
#
# model_saved = LightweightCNN()
# model_saved.load_state_dict(torch.load(os.path.join(RESULT_DIRECTORY, "initial_model_t&val_080101split.pth")))
# model_saved.to(device)
# model_saved.eval()

RuntimeError: Error(s) in loading state_dict for SimpleCNN:
	Missing key(s) in state_dict: "bn1.weight", "bn1.bias", "bn1.running_mean", "bn1.running_var", "bn2.weight", "bn2.bias", "bn2.running_mean", "bn2.running_var", "bn3.weight", "bn3.bias", "bn3.running_mean", "bn3.running_var". 